In [1]:
# Adapted from https://quantumcomputinguk.org/tutorials/grovers-algorithm-with-code
# Based on A fast quantum mechanical algorithm for database search by L. Grover (1996)

# Takes integer input with bit length <= 4 from user
# Forms quantum circuit with 4 qubits and 4 classical bits
# Applies oracle based on integer input
# Applies amplitude amplification
# Measures, then returns results
# Result with greates amount of hits is used as answer
# Answer returned with accuracy

In [2]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, IBMQ
import math
from time import time
from math import pi
%matplotlib inline

In [3]:
def oracle_0(qc, q):
    # 0000 Oracle

    qc.x(q[0])
    qc.x(q[1])
    qc.x(q[2])
    qc.x(q[3])

    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

    qc.x(q[0])
    qc.x(q[1])
    qc.x(q[2])
    qc.x(q[3])

In [4]:
def oracle_1(qc, q):
    # 0001 Oracle

    qc.x(q[1])
    qc.x(q[2])
    qc.x(q[3])

    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

    qc.x(q[1])
    qc.x(q[2])
    qc.x(q[3])

In [5]:
def oracle_2(qc, q):
    # 0010 Oracle

    qc.x(q[0])
    qc.x(q[2])
    qc.x(q[3])

    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

    qc.x(q[0])
    qc.x(q[2])
    qc.x(q[3])

In [6]:
def oracle_3(qc, q):
    # 0011 Oracle

    qc.x(q[2])
    qc.x(q[3])

    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

    qc.x(q[2])
    qc.x(q[3])

In [7]:
def oracle_4(qc, q):
    # 0100 Oracle

    qc.x(q[0])
    qc.x(q[1])
    qc.x(q[3])

    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

    qc.x(q[0])
    qc.x(q[1])
    qc.x(q[3])

In [8]:
def oracle_5(qc, q):
    # 0101 Oracle

    qc.x(q[1])
    qc.x(q[3])

    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

    qc.x(q[1])
    qc.x(q[3])

In [9]:
def oracle_6(qc, q):
    # 0110 Oracle

    qc.x(q[0])
    qc.x(q[3])

    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

    qc.x(q[0])
    qc.x(q[3])

In [10]:
def oracle_7(qc, q):
    # 0111 Oracle

    qc.x(q[3])

    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

    qc.x(q[3])

In [11]:
def oracle_8(qc, q):
    # 1000 Oracle

    qc.x(q[0])
    qc.x(q[1])
    qc.x(q[2])

    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

    qc.x(q[0])
    qc.x(q[1])
    qc.x(q[2])

In [12]:
def oracle_9(qc, q):
    # 1001 Oracle

    qc.x(q[1])
    qc.x(q[2])

    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

    qc.x(q[1])
    qc.x(q[2])

In [13]:
def oracle_10(qc, q):
    # 1010 Oracle

    qc.x(q[0])
    qc.x(q[2])

    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

    qc.x(q[0])
    qc.x(q[2])

In [14]:
def oracle_11(qc, q):
    # 1011 Oracle

    qc.x(q[3])

    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

    qc.x(q[3])

In [15]:
def oracle_12(qc, q):
    # 1100 Oracle

    qc.x(q[0])
    qc.x(q[1])

    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

    qc.x(q[0])
    qc.x(q[1])

In [16]:
def oracle_13(qc, q):
    # 1101 Oracle 

    qc.x(q[1])

    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

    qc.x(q[1])

In [17]:
def oracle_14(qc, q):
    # 1110 Oracle

    qc.x(q[0])

    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

    qc.x(q[0])

In [18]:
def oracle_15(qc, q):
    # 1111 Oracle
    
    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

In [19]:
def oracle_ctl(tgt, qc, q):
    # Runs oracle depending on the target of the search
    if tgt==0:
        oracle_0(qc, q)
    if tgt==1:
        oracle_1(qc, q)
    if tgt==2:
        oracle_2(qc, q)
    if tgt==3:
        oracle_3(qc, q)
    if tgt==4:
        oracle_4(qc, q)
    if tgt==5:
        oracle_5(qc, q)
    if tgt==6:
        oracle_6(qc, q)
    if tgt==7:
        oracle_7(qc, q)
    if tgt==8:
        oracle_8(qc, q)
    if tgt==9:
        oracle_9(qc, q)
    if tgt==10:
        oracle_10(qc, q)
    if tgt==11:
        oracle_11(qc, q)
    if tgt==12:
        oracle_12(qc, q)
    if tgt==13:
        oracle_13(qc, q)
    if tgt==14:
        oracle_14(qc, q)
    if tgt==15:
        oracle_15(qc,q)

In [20]:
def grover(N):
    time_i = time()
    
    # Step 1
    # Initialize circuit
        
    pi = math.pi
    q = QuantumRegister(4,'q')
    c = ClassicalRegister(4,'c')
    qc = QuantumCircuit(q,c)

    for i in range(4):
        qc.h(q[i]) 
        
    # Step 2
    # Apply operators Uw and Us to circuit
    
    # Operator Uw aka the oracle
    oracle_ctl(N, qc, q)

    # Operator Us aka amplitude amplification
    for i in range(4):
        qc.h(q[i])
        qc.x(q[i])

    qc.cu1(pi/4, q[0], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(-pi/4, q[1], q[3])
    qc.cx(q[0], q[1])
    qc.cu1(pi/4, q[1], q[3])
    qc.cx(q[1], q[2])
    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])
    qc.cx(q[1], q[2])

    qc.cu1(-pi/4, q[2], q[3])
    qc.cx(q[0], q[2])
    qc.cu1(pi/4, q[2], q[3])

    for i in range(4):
        qc.x(q[i])
        qc.h(q[i])
    
    # Step 3
    # Measurement
    qc.barrier(q)
    for i in range(4):
        qc.measure(q[i], c[i])

    # Simulate circuit
    backend = provider.get_backend('ibmq_qasm_simulator')
    print('\n\tSimulating circuit...\n')
    job = execute(qc, backend, shots=100)
    counts = job.result().get_counts()

    maxval=0

    for i in counts.keys():
        if counts[i] > maxval:
            maxval=counts[i]
            x=i
        
    time_f = time()
    
    time_delta = time_f - time_i

    print("\n\tFound %i in %i/%i shots" % (int(x,2), maxval, 100))
    print("\tAlgorithm took: %f seconds" % time_delta)
    
    return time_delta

In [22]:
print("--------------------")
print("Appendix B")
print("Grover's Quantum Search Algorithm\n\n")

# Qiskit details to choose provider and load account
print("Loading IBMQ account...")
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')
    
# Target that user wishes to find
N = int(input('Enter 4 bit number: '))

# Initialize variables
times = []
sums = 0
runs = int(input("How many times to run: "))

# Runs factoring algorithm a number of times
# Gets total amount of time
# Saves individual time per run for later
for i in range(runs):
    print("\n\n****************\nRun # %i" % (i+1))
    time_to_run = grover(N)
    sums += time_to_run
    times.append(time_to_run)

# Average amount of time per run
avg = sums/runs
# Sum of deviations from average
devsum = 0

for item in times:
    # Calculates square of each deviation
    dev = (avg - item)**2
    # Stores sum of squares
    devsum += dev

# Gets the square of the average of the deviations
# Calculates standard deviation
avg_sq = devsum/runs    
standard_dev = avg_sq**(1/2) 


print("\nAverage time per run: %f" % (avg))
print("Standard Deviation: %f" % standard_dev)

print("Done.")
print("--------------------")



--------------------
Appendix B
Grover's Quantum Search Algorithm


Loading IBMQ account...


ibmqfactory.load_account:WARNING:2020-08-18 19:37:45,126: Credentials are already in use. The existing account in the session will be replaced.


Enter 4 bit number: 15
How many times to run: 100


****************
Run # 1

	Simulating circuit...


	Found 15 in 56/100 shots
	Algorithm took: 6.980801 seconds


****************
Run # 2

	Simulating circuit...


	Found 15 in 48/100 shots
	Algorithm took: 7.155693 seconds


****************
Run # 3

	Simulating circuit...


	Found 15 in 45/100 shots
	Algorithm took: 5.671464 seconds


****************
Run # 4

	Simulating circuit...


	Found 15 in 42/100 shots
	Algorithm took: 6.996946 seconds


****************
Run # 5

	Simulating circuit...


	Found 15 in 43/100 shots
	Algorithm took: 6.272846 seconds


****************
Run # 6

	Simulating circuit...


	Found 15 in 51/100 shots
	Algorithm took: 7.151806 seconds


****************
Run # 7

	Simulating circuit...


	Found 15 in 46/100 shots
	Algorithm took: 6.330433 seconds


****************
Run # 8

	Simulating circuit...


	Found 15 in 46/100 shots
	Algorithm took: 6.192654 seconds


****************
Run # 9

	Simulating circui


	Found 15 in 50/100 shots
	Algorithm took: 6.149942 seconds


****************
Run # 74

	Simulating circuit...


	Found 15 in 40/100 shots
	Algorithm took: 6.053681 seconds


****************
Run # 75

	Simulating circuit...


	Found 15 in 54/100 shots
	Algorithm took: 6.771904 seconds


****************
Run # 76

	Simulating circuit...


	Found 15 in 42/100 shots
	Algorithm took: 6.418760 seconds


****************
Run # 77

	Simulating circuit...


	Found 15 in 41/100 shots
	Algorithm took: 6.654567 seconds


****************
Run # 78

	Simulating circuit...


	Found 15 in 55/100 shots
	Algorithm took: 5.824521 seconds


****************
Run # 79

	Simulating circuit...


	Found 15 in 44/100 shots
	Algorithm took: 6.903708 seconds


****************
Run # 80

	Simulating circuit...


	Found 15 in 50/100 shots
	Algorithm took: 5.941601 seconds


****************
Run # 81

	Simulating circuit...


	Found 15 in 49/100 shots
	Algorithm took: 6.820211 seconds


****************
Run # 82